1. Why proballity of object creator is to "close" to 0.5

In [ ]:
bandDict={0: 'maskBand',
 1: 'B01new',
 2: 'B02new',
 3: 'B03new',
 4: 'B04new',
 5: 'B05new',
 6: 'B06new',
 7: 'B07new',
 8: 'B08new',
 9: 'B8Anew',
 10: 'B09new',
 11: 'B10new',
 12: 'B11new',
 13: 'B12new',
 14: 'B01old',
 15: 'B02old',
 16: 'B03old',
 17: 'B04old',
 18: 'B05old',
 19: 'B06old',
 20: 'B07old',
 21: 'B08old',
 22: 'B8Aold',
 23: 'B09old',
 24: 'B10old',
 25: 'B11old',
 26: 'B12old'}

In [4]:
#import imgaug as ia
from imgaug import augmenters as iaa
from skimage import io

import glob
import numpy as np

import tensorflow as tf
from tensorflow.keras.callbacks import (EarlyStopping, 
                                        ModelCheckpoint, 
                                        ReduceLROnPlateau, 
                                        CSVLogger, 
                                        TensorBoard)

In [95]:
class DataGenerator(tf.keras.utils.Sequence):
    """
    Custom data generator for Unet PermForestChange
    
    Main parameter is unputTiffs - list to path to tiff files
    
    Sample usage 
    allTifs = list(glob.glob('../../ds/tiles_256_256_27/*/*.tif'))

    validSize = 250
    
    train_generator = DataGenerator(allTifs[:-validSize],batch_size=batch)
    
    """
    def __init__(self, inputTiffs, batch_size=32, num_classes=2, shuffle=True):
        self.inputTiffs = inputTiffs
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        #custom augmentation with imgaug lib
        self.aug_pipe = iaa.Sequential(
            [
                iaa.SomeOf((0,1),
                [
                    #iaa.Add((-30, -10)), # change brightness of images (by -10 to 10 of original value)
                    iaa.Multiply((0.8, 1.2)),
                    
                    # Geometric transfroms
                    iaa.Rotate((-45,45)),
                    iaa.TranslateX(px=(-20, 20)),
                    iaa.TranslateY(px=(-20, 20)),
                ]
                ),
            ],
            random_order=True
        )

    def __len__(self):
        return len(self.inputTiffs) // self.batch_size

    def __getitem__(self, index):
        indexes = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        #print (indexes)
        batchTifs = [self.inputTiffs[k] for k in indexes]
        
        X, y = self._get_data(batchTifs)
        return X, y

    def on_epoch_end(self):
        self.index = np.arange(len(self.inputTiffs))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def _get_data(self, batch):
        #X = inArray
        #y = # logic
        
        X = []
        y = []
        
        for idx,imagePath in enumerate(batch):
            
            #print (image)
            
            # raw image
            img = io.imread(imagePath)
            
            # flip with cv2
            #flip = np.random.binomial(1, .5)
            #if flip > 0.5:
                #img = self.flip(img)
            
            # aug with imgaug lib
            img = self.aug_pipe.augment_image(img)
            
            # x with features
            # channel indexes
            # mask - 0 
            # NEW
            #["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"]
            #[ 1,     2,    3,    4,    5,    6,    7,    8,    9,    10,   11,   12,   13  ]
            # OLD
            #["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"]
            #[ 14,    15,   16,   17,   18,   19,   20,   21,   22,   23,   24,   25,   26  ]
            X.append(np.array(
                np.dstack([
                    #img[:,:,17],# old red
                    #img[:,:,4], # new red
                    img[:,:,4]/23779 - img[:,:,17]/28000, # dif
                    #img[:,:,8],
                    #img[:,:,21],
                    img[:,:,8]/24796 - img[:,:,21]/28002,
                    #(
                    #    (img[:,:,21] - img[:,:,17])/(img[:,:,21] + img[:,:,17])- # old ndvi
                    #    (img[:,:,8] - img[:,:,4]) /(img[:,:,8] + img[:,:,4])     # new ndvi
                    #   
                    #), # ndvi diff
                    #img[:,:,13],
                    #img[:,:,26],
                    img[:,:,13]/28000 - img[:,:,26]/28000,
                    
                ])).astype('float32'))
            
            # to one class all changes
            y_value = None
            if "_with_object.tif" in imagePath:
                y_value = 1
            else :
                y_value = 0
            
            y.append(y_value)
            #print(np.array(y).shape)
            
        return np.array(X), np.array(y)
    
    def flip(self,image):
        """
        Random flip image around x, y or both axes
        image is np.array
        """
        flipDirection = np.random.randint(-1,1,1)[0]
        return cv2.flip(image,flipDirection)
        

In [96]:
model  = tf.keras.applications.VGG16(
    include_top=False, 
    input_tensor=None, 
    input_shape=(256,256,3),
    pooling=None, 
    classes=1000, 
    classifier_activation='softmax',
    weights='./files/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
)

# Res net

In [98]:
conv_base = tf.keras.applications.ResNet50(weights='./files/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                  include_top=False,
                  input_shape=(256, 256, 3))

conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers[:100]:
    layer.trainable = False
conv_base.summary()

model = tf.keras.models.Sequential()
model.add(conv_base)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dropout (Dropout)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               33554688  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 57,142,657
Trainable params: 53,007,873
Non-trainable params: 4,134,784
_________________________________________________________________


In [99]:
callbacks = [
          ModelCheckpoint("files/modelIsObjectOnTile.h5"),
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
          CSVLogger("files/dataIsObjectOnTile.csv"),
]


opt = tf.keras.optimizers.RMSprop(lr=2e-5)
metrics = ["acc"]
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)

batch = 16

epochs = 5

#train_steps = 3405-100//batch
#valid_steps = 100//batch

#if len(X_train) % batch != 0:
    #train_steps += 1
#if len(X_test) % batch != 0:
    #valid_steps += 1

allTifs = list(glob.glob('../../ds/tiles_256_256_27/*/*.tif'))

validSize = 250
    
train_generator = DataGenerator(allTifs[:-validSize],batch_size=batch)
valid_generator = DataGenerator(allTifs[-validSize:],batch_size=batch)

model.fit(train_generator,
                    steps_per_epoch=len(allTifs[:-validSize])//batch,
                    validation_data=valid_generator,
                    epochs=epochs,
                    validation_steps=len(allTifs[-validSize:])//batch,
                    callbacks=callbacks)

Epoch 1/5
179/179 [==============================] - 266s 1s/step - loss: 1.2461 - acc: 0.5017 - val_loss: 0.8894 - val_acc: 0.5125 - lr: 2.0000e-05
Epoch 2/5
179/179 [==============================] - 263s 1s/step - loss: 0.8455 - acc: 0.4920 - val_loss: 0.7751 - val_acc: 0.5708 - lr: 2.0000e-05
Epoch 3/5
179/179 [==============================] - 263s 1s/step - loss: 0.7426 - acc: 0.5003 - val_loss: 0.7331 - val_acc: 0.5708 - lr: 2.0000e-05
Epoch 4/5
179/179 [==============================] - 262s 1s/step - loss: 0.7356 - acc: 0.5021 - val_loss: 0.6901 - val_acc: 0.5792 - lr: 2.0000e-05
Epoch 5/5
179/179 [==============================] - 266s 1s/step - loss: 0.7081 - acc: 0.5150 - val_loss: 0.7748 - val_acc: 0.4083 - lr: 2.0000e-05


In [100]:
from matplotlib import pyplot as plt

#testImg = '../../ds/tiles_256_256_27_test/T39VVJ_pair_0.tif_8968_5479_no_object.tif'
toPredict = []
for imgTiff in glob.glob ('../../ds/tiles_256_256_27_test/*.tif'):
    img = io.imread(imgTiff)/65536

                #flip = np.random.binomial(1, .5)
                #if flip > 0.5:
                #    img = cv2.flip(img, 1)

                # x with features
    toPredict.append( np.array(
                    np.dstack([
                        img[:,:,4] - img[:,:,17], # dif
                        #img[:,:,8],
                        #img[:,:,21],
                        img[:,:,8] - img[:,:,21],
                        #(
                        #    (img[:,:,21] - img[:,:,17])/(img[:,:,21] + img[:,:,17])- # old ndvi
                        #    (img[:,:,8] - img[:,:,4]) /(img[:,:,8] + img[:,:,4])     # new ndvi
                        #    
                        #), # ndvi diff
                        #img[:,:,13],
                        #img[:,:,26],
                        img[:,:,13] - img[:,:,26],

                ]).astype('float32')))

#testPredcition = model.predict(np.array([toPredict]))

#display output
#fig,axes = plt.subplots(ncols=2,figsize=(15,15))
#axes[0].imshow(testPredcition[0].astype('float32'))
#axes[1].imshow(img[:,:,0])
predictedArray = model.predict(np.array(toPredict))

In [14]:
print (predictedArray.shape)
print (np.array(toPredict).shape)

(286, 1)
(286, 256, 256, 3)


In [93]:
def calcMaxValues(inTiffMask='../../ds/tiles_256_256_27_test/*.tif'):
    # mask - 0 
    # NEW
    #["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"]
    #[ 1,     2,    3,    4,    5,    6,    7,    8,    9,    10,   11,   12,   13  ]
    # OLD
    #["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"]
    #[ 14,    15,   16,   17,   18,   19,   20,   21,   22,   23,   24,   25,   26  ]
    
    maxs = []
    for imgTiff in glob.glob(inTiffMask):
        img = io.imread(imgTiff)
        maxsBandValues = np.max(img.reshape((img.shape[0]*img.shape[1],img.shape[2])),axis=0)
        maxs.append(maxsBandValues)
    maxs = np.max(np.array(maxs),axis=0)
    
    for bandIdx in bandsDict.keys():
        print(f'{bandIdx} - ({bandsDict[bandIdx]}) max value {maxs[bandIdx]}')
#calcMaxValues()

In [94]:
calcMaxValues(inTiffMask='../../ds/tiles_256_256_27/*/*.tif')

0 - (maskBand) max value 4
1 - (B01new) max value 10163
2 - (B02new) max value 22843
3 - (B03new) max value 20625
4 - (B04new) max value 23779
5 - (B05new) max value 21645
6 - (B06new) max value 22797
7 - (B07new) max value 23368
8 - (B08new) max value 24796
9 - (B8Anew) max value 23897
10 - (B09new) max value 9759
11 - (B10new) max value 1881
12 - (B11new) max value 28000
13 - (B12new) max value 28000
14 - (B01old) max value 10270
15 - (B02old) max value 28000
16 - (B03old) max value 28000
17 - (B04old) max value 28000
18 - (B05old) max value 24759
19 - (B06old) max value 26785
20 - (B07old) max value 27263
21 - (B08old) max value 28002
22 - (B8Aold) max value 28000
23 - (B09old) max value 9110
24 - (B10old) max value 1731
25 - (B11old) max value 24148
26 - (B12old) max value 28000


In [101]:
predictedArray

array([[0.6251441 ],
       [0.62520385],
       [0.62514013],
       [0.6251287 ],
       [0.62553155],
       [0.62516   ],
       [0.6252359 ],
       [0.62514406],
       [0.62521243],
       [0.62511957],
       [0.6251584 ],
       [0.6249953 ],
       [0.6252962 ],
       [0.6251849 ],
       [0.62515223],
       [0.6250522 ],
       [0.6251418 ],
       [0.6252138 ],
       [0.6252393 ],
       [0.6252299 ],
       [0.6252088 ],
       [0.62546873],
       [0.625134  ],
       [0.6252367 ],
       [0.62518096],
       [0.62517655],
       [0.625145  ],
       [0.6251684 ],
       [0.62518907],
       [0.6251509 ],
       [0.62513816],
       [0.62522376],
       [0.6251449 ],
       [0.6250989 ],
       [0.62516373],
       [0.6251414 ],
       [0.6251684 ],
       [0.6251552 ],
       [0.625166  ],
       [0.62516654],
       [0.62516093],
       [0.6251888 ],
       [0.6251774 ],
       [0.62520087],
       [0.6252358 ],
       [0.62520945],
       [0.62514025],
       [0.625

In [103]:
testTifs = list(glob.glob('../../ds/tiles_256_256_27_test/*/*.tif'))

test_generator = DataGenerator(testTifs,batch_size=16)
test_loss, test_acc = model.evaluate(test_generator)
print('test acc:', test_acc)

AttributeError: 'NoneType' object has no attribute 'update'